d-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 400px">
</div>

# DataFrame & Column
1. Construct columns
1. Subset columns
1. Add or replace columns
1. Subset rows
1. Sort rows

##### Methods
- DataFrame (<a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframe#pyspark.sql.DataFrame" target="_blank">Python</a>/<a href="http://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/Dataset.html" target="_blank">Scala</a>): `select`, `selectExpr`, `drop`, `withColumn`, `withColumnRenamed`, `filter`, `distinct`, `limit`, `sort`
- Column (<a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=column#pyspark.sql.Column" target="_blank">Python</a>/<a href="http://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/Column.html" target="_blank">Scala</a>): `alias`, `isin`, `cast`, `isNotNull`, `desc`, operators

In [0]:
%run ./Includes/Classroom-Setup

Let's use the BedBricks events dataset.

In [0]:
eventsDF = spark.read.parquet(eventsPath)
display(eventsDF)

### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Construct columns

A **column** is a logical construction that will be computed based on the data in a DataFrame using an expression

Construct a new column based on the input columns existing in a DataFrame

In [0]:
from pyspark.sql.functions import col

col("device")
eventsDF.device
eventsDF["device"]

Use column objects to form complex expressions

In [0]:
col("ecommerce.purchase_revenue_in_usd") + col("ecommerce.total_item_quantity")
col("event_timestamp").desc()
(col("ecommerce.purchase_revenue_in_usd") * 100).cast("int")

### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Subset columns
Use DataFrame transformations to subset columns

#### **`select()`**
Selects a set of columns or column based expressions

In [0]:
devicesDF = eventsDF.select("user_id", "device")
display(devicesDF)

In [0]:
from pyspark.sql.functions import col

locationsDF = eventsDF.select("user_id", 
  col("geo.city").alias("city"),
  col("geo.state").alias("state"))

display(locationsDF)

#### **`selectExpr()`**
Selects a set of SQL expressions

In [0]:
appleDF = eventsDF.selectExpr("user_id", "device in ('macOS', 'iOS') as apple_user")
display(appleDF)

#### `drop()`
Returns a new DataFrame after dropping the given column, specified as a string or column object

Use strings to specify multiple columns

In [0]:
anonymousDF = eventsDF.drop("user_id", "geo", "device")
display(anonymousDF)

In [0]:
noSalesDF = eventsDF.drop(col("ecommerce"))
display(noSalesDF)

### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Add or replace columns
Use DataFrame transformations to add or replace columns

#### `withColumn`
Returns a new DataFrame by adding a column or replacing the existing column that has the same name.

In [0]:
mobileDF = eventsDF.withColumn("mobile", col("device").isin("iOS", "Android"))
display(mobileDF)

In [0]:
purchaseQuantityDF = eventsDF.withColumn("purchase_quantity", col("ecommerce.total_item_quantity").cast("int"))
purchaseQuantityDF.printSchema()

#### `withColumnRenamed()`
Returns a new DataFrame with a column renamed.

In [0]:
locationDF = eventsDF.withColumnRenamed("geo", "location")
display(locationDF)

### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Subset Rows
Use DataFrame transformations to subset rows

#### `filter()`
Filters rows using the given SQL expression or column based condition.

In [0]:
purchasesDF = eventsDF.filter("ecommerce.total_item_quantity > 0")
display(purchasesDF)

In [0]:
revenueDF = eventsDF.filter(col("ecommerce.purchase_revenue_in_usd").isNotNull())
display(revenueDF)

In [0]:
androidDF = eventsDF.filter((col("traffic_source") != "direct") & (col("device") == "Android"))
display(androidDF)

#### `dropDuplicates()`
Returns a new DataFrame with duplicate rows removed, optionally considering only a subset of columns.

##### Alias: `distinct`

In [0]:
eventsDF.distinct()

In [0]:
distinctUsersDF = eventsDF.dropDuplicates(["user_id"])
display(distinctUsersDF)

#### `limit()`
Returns a new DataFrame by taking the first n rows.

In [0]:
limitDF = eventsDF.limit(100)
display(limitDF)

### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Sort rows
Use DataFrame transformations to sort rows

#### `sort()`
Returns a new DataFrame sorted by the given columns or expressions.

##### Alias: `orderBy`

In [0]:
increaseTimestampsDF = eventsDF.sort("event_timestamp")
display(increaseTimestampsDF)

In [0]:
decreaseTimestampsDF = eventsDF.sort(col("event_timestamp").desc())
display(decreaseTimestampsDF)

In [0]:
increaseSessionsDF = eventsDF.orderBy(["user_first_touch_timestamp", "event_timestamp"])
display(increaseSessionsDF)

In [0]:
decreaseSessionsDF = eventsDF.sort(col("user_first_touch_timestamp").desc(), col("event_timestamp"))
display(decreaseSessionsDF)

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Purchase Revenues Lab

Prepare dataset of events with purchase revenue.
1. Extract purchase revenue for each event
2. Filter events where revenue is not null
3. Check what types of events have revenue
4. Drop unneeded column

##### Methods
- DataFrame: `select`, `drop`, `withColumn`, `filter`, `dropDuplicates`
- Column: `isNotNull`

In [0]:
eventsDF = spark.read.parquet(eventsPath)
display(eventsDF)

### 1. Extract purchase revenue for each event
Add new column **`revenue`** by extracting **`ecommerce.purchase_revenue_in_usd`**

In [0]:
# TODO
revenueDF = eventsDF.FILL_IN
display(revenueDF)

-sandbox
##### <img alt="Best Practice" title="Best Practice" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-blue-ribbon.svg"/> Check your work

In [0]:
expected1 = [5830.0, 5485.0, 5289.0, 5219.1, 5180.0, 5175.0, 5125.0, 5030.0, 4985.0, 4985.0]
result1 = [row.revenue for row in revenueDF.sort(col("revenue").desc_nulls_last()).limit(10).collect()]

assert(expected1 == result1)

### 2. Filter events where revenue is not null
Filter for records where **`revenue`** is not **`null`**

In [0]:
# TODO
purchasesDF = revenueDF.FILL_IN
display(purchasesDF)

### 3. Check what types of events have revenue
Find unique **`event_name`** values in **`purchasesDF`** in one of two ways:
- Select "event_name" and get distinct records
- Drop duplicate records based on the "event_name" only

Hint: There's only one event associated with revenues

In [0]:
# TODO
distinctDF = purchasesDF.FILL_IN
display(distinctDF)

### 4. Drop unneeded column
Since there's only one event type, drop **`event_name`** from **`purchasesDF`**.

In [0]:
# TODO
finalDF = purchasesDF.FILL_IN
display(finalDF)

### 5. Chain all the steps above excluding step 3

In [0]:
# TODO
finalDF = (eventsDF
  .FILL_IN
)

display(finalDF)

-sandbox
##### <img alt="Best Practice" title="Best Practice" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-blue-ribbon.svg"/> Check your work

In [0]:
assert(finalDF.count() == 180678)

In [0]:
expected_columns = {'device', 'ecommerce', 'event_previous_timestamp', 'event_timestamp', 
                    'geo', 'items', 'revenue', 'traffic_source', 
                    'user_first_touch_timestamp', 'user_id'}
assert(set(finalDF.columns) == expected_columns)

### Clean up classroom

In [0]:
%run ./Includes/Classroom-Cleanup
